In [1]:
from pyspark.sql import SparkSession, DataFrame, Window
import pyspark.sql.functions as F
import pandas as pd

In [2]:
import getpass
username = getpass.getuser()

username

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '4000'). \
    config("spark.sql.warehouse.dir", "hdfs://0.0.0.0:9000/user/hive/warehouse/"). \
    config("spark.sql.debug.maxToStringFields", 1000). \
    config("spark.dynamicAllocation.enabled", "false"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Data Processing - Overview'). \
    master('yarn'). \
    getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
orders = spark.read.json('/home/nghiaht7/data-engineer/data-engineering-essentials/data/retail_db_json/orders')

In [4]:
order_items = spark.read.json('/home/nghiaht7/data-engineer/data-engineering-essentials/data/retail_db_json/order_items')

In [5]:
orders.printSchema()

root
 |-- order_customer_id: long (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_id: long (nullable = true)
 |-- order_status: string (nullable = true)



In [6]:
order_items.printSchema()

root
 |-- order_item_id: long (nullable = true)
 |-- order_item_order_id: long (nullable = true)
 |-- order_item_product_id: long (nullable = true)
 |-- order_item_product_price: double (nullable = true)
 |-- order_item_quantity: long (nullable = true)
 |-- order_item_subtotal: double (nullable = true)



# inner join


In [11]:
orders_join = orders.join(
    order_items, 
    orders.order_id == order_items.order_item_order_id
)

orders_join.toPandas().head(10)

,order_customer_id,order_date,order_id,order_status,order_item_id,order_item_order_id,order_item_product_id,order_item_product_price,order_item_quantity,order_item_subtotal
0,11599,2013-07-25 00:00:00.0,1,CLOSED,1,1,957,299.98,1,299.98
1,256,2013-07-25 00:00:00.0,2,PENDING_PAYMENT,2,2,1073,199.99,1,199.99
2,256,2013-07-25 00:00:00.0,2,PENDING_PAYMENT,3,2,502,50.00,5,250.00
3,256,2013-07-25 00:00:00.0,2,PENDING_PAYMENT,4,2,403,129.99,1,129.99
4,8827,2013-07-25 00:00:00.0,4,CLOSED,5,4,897,24.99,2,49.98
5,8827,2013-07-25 00:00:00.0,4,CLOSED,6,4,365,59.99,5,299.95
6,8827,2013-07-25 00:00:00.0,4,CLOSED,7,4,502,50.00,3,150.00
7,8827,2013-07-25 00:00:00.0,4,CLOSED,8,4,1014,49.98,4,199.92
8,11318,2013-07-25 00:00:00.0,5,COMPLETE,9,5,957,299.98,1,299.98
9,11318,2013-07-25 00:00:00.0,5,COMPLETE,10,5,365,59.99,5,299.95


In [ ]:
# orders_join = orders.join(
#     order_items, 
#     on=orders.order_id == order_items.order_item_order_id
# )

In [ ]:
orders_join = orders.join(
    order_items, 
    on=orders['order_id'] == order_items['order_item_order_id']
)

In [ ]:
# orders_join = orders.join(
#     order_items, 
#     on=orders['order_id'] == order_items['order_item_order_id'],
#     how='inner'
# )

In [12]:
# select(orders.order_id, orders.order_date, orders.order_status, order_items.order_item_subtotal). \

# select(orders['order_id'], orders['order_date'], orders['order_status'], order_items['order_item_subtotal']). \


orders. \
    join(
        order_items, 
        on=orders['order_id'] == order_items['order_item_order_id'],
        how='inner'
    ). \
    select(orders['*'], order_items['order_item_subtotal']). \
    show()

+-----------------+--------------------+--------+---------------+-------------------+
|order_customer_id|          order_date|order_id|   order_status|order_item_subtotal|
+-----------------+--------------------+--------+---------------+-------------------+
|            11599|2013-07-25 00:00:...|       1|         CLOSED|             299.98|
|              256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|             199.99|
|              256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|              250.0|
|              256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|             129.99|
|             8827|2013-07-25 00:00:...|       4|         CLOSED|              49.98|
|             8827|2013-07-25 00:00:...|       4|         CLOSED|             299.95|
|             8827|2013-07-25 00:00:...|       4|         CLOSED|              150.0|
|             8827|2013-07-25 00:00:...|       4|         CLOSED|             199.92|
|            11318|2013-07-25 00:00:...|       5|     

In [13]:
airportCodesPath = "/home/nghiaht7/data-engineer/data-engineering-essentials/data/airport-codes.csv"

In [14]:
airportCodes = spark.read.format("csv"). \
            option("header", "true"). \
            option("inferSchema", "true"). \
            load(airportCodesPath)

In [15]:
airportCodes.printSchema()

root
 |-- IATA_CODE: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)



In [16]:
airportCodes.show(10)

+---------+-------------+-----+-------+
|IATA_CODE|         CITY|STATE|COUNTRY|
+---------+-------------+-----+-------+
|      ABE|    Allentown|   PA|    USA|
|      ABI|      Abilene|   TX|    USA|
|      ABQ|  Albuquerque|   NM|    USA|
|      ABR|     Aberdeen|   SD|    USA|
|      ABY|       Albany|   GA|    USA|
|      ACK|    Nantucket|   MA|    USA|
|      ACT|         Waco|   TX|    USA|
|      ACV|Arcata/Eureka|   CA|    USA|
|      ACY|Atlantic City|   NJ|    USA|
|      ADK|         Adak|   AK|    USA|
+---------+-------------+-----+-------+
only showing top 10 rows



In [17]:
airtraffic_path = '/home/nghiaht7/data-engineer/data-engineering-essentials/data/airflights_delay'

In [18]:
airtraffic = spark.read.format("csv"). \
            option("header", "true"). \
            option("inferSchema", "true"). \
            load(airtraffic_path)

In [19]:
airtraffic.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- 

In [21]:
airtraffic.select("Origin").toPandas().head(5)

,Origin
0,IAD
1,IAD
2,IND
3,IND
4,IND


# * Get number of flights departed from each of the US airport.

In [28]:
result = airtraffic.join(airportCodes, airtraffic.Origin == airportCodes.IATA_CODE).\
    select(airportCodes["*"], airtraffic.Origin). \
    groupBy("Origin").agg(F.count(F.lit(1)).alias("FlightCount")).orderBy(F.col("FlightCount").desc())

result.show()

+------+-----------+
|Origin|FlightCount|
+------+-----------+
|   LAS|       6817|
|   MDW|       6332|
|   PHX|       5568|
|   BWI|       4578|
|   OAK|       3949|
|   HOU|       3825|
|   DAL|       3525|
|   LAX|       3369|
|   SAN|       3344|
|   MCO|       3310|
|   SMF|       2656|
|   TPA|       2387|
|   BNA|       2286|
|   ONT|       2275|
|   MCI|       2218|
|   SJC|       2197|
|   ABQ|       2040|
|   STL|       2036|
|   PHL|       1714|
|   SAT|       1649|
+------+-----------+
only showing top 20 rows



# * Get number of flights departed from each of the state.

In [29]:
airtraffic.join(airportCodes, airtraffic.Origin == airportCodes.IATA_CODE).\
    select(airportCodes["*"], airtraffic.Origin). \
    groupBy("State").agg(F.count(F.lit(1)).alias("FlightCount")).orderBy(F.col("FlightCount").desc())

State,FlightCount
CA,21657
TX,13034
FL,8393
NV,8089
AZ,6462
IL,6333
MD,4578
MO,4254
PA,2357
TN,2286


# * Get the list of airports in the US from which flights are not departed.

* This is an example for outer join.
* We need to get those airports which are in airport codes but not in 2008 January airtraffic data set.
* Based on the side of the airport codes data set, we can say left or right. We will invoke join using airport codes data set and hence we will use left outer join.

In [33]:
airportCodes. \
    join(airtraffic, F.col("IATA_CODE") == F.col("Origin"), "left"). \
    filter("Origin IS NULL"). \
    select(airportCodes["*"], F.col("Origin")). \
    show()

+---------+-------------+-----+-------+------+
|IATA_CODE|         CITY|STATE|COUNTRY|Origin|
+---------+-------------+-----+-------+------+
|      ABE|    Allentown|   PA|    USA|  null|
|      ABI|      Abilene|   TX|    USA|  null|
|      ABR|     Aberdeen|   SD|    USA|  null|
|      ABY|       Albany|   GA|    USA|  null|
|      ACK|    Nantucket|   MA|    USA|  null|
|      ACT|         Waco|   TX|    USA|  null|
|      ACV|Arcata/Eureka|   CA|    USA|  null|
|      ACY|Atlantic City|   NJ|    USA|  null|
|      ADK|         Adak|   AK|    USA|  null|
|      ADQ|       Kodiak|   AK|    USA|  null|
|      AEX|   Alexandria|   LA|    USA|  null|
|      AGS|      Augusta|   GA|    USA|  null|
|      AKN|  King Salmon|   AK|    USA|  null|
|      ALO|     Waterloo|   IA|    USA|  null|
|      ANC|    Anchorage|   AK|    USA|  null|
|      APN|       Alpena|   MI|    USA|  null|
|      ASE|        Aspen|   CO|    USA|  null|
|      ATL|      Atlanta|   GA|    USA|  null|
|      ATW|  

In [34]:
airportCodes. \
    join(airtraffic, F.col("IATA_CODE") == F.col("Origin"), "left"). \
    filter("Origin IS NULL"). \
    select(airportCodes["*"], F.col("Origin")). \
    count()

240

# * Check if there are any origins in airtraffic data which do not have record in airport-codes.


* This is an example for outer join.
* We need to get those airports which are in Origin field in January 2008 airtraffic data set but not in airport-codes. We need to consider all the valid records from airport codes.
* Based on the side of the airtraffic data set, we can say left or right. We will be invoking join using airtraffic data set and hence we will use left outer join.
* We will also apply distinct on Origin before performing left outer join.

In [42]:
airtraffic.select("Origin").distinct(). \
    join(airportCodes, F.col("Origin") == F.col("IATA_CODE"), "left"). \
    select(airportCodes["*"], F.col("Origin")). \
    filter("IATA_CODE IS NULL"). \
    count()

0

# * Get the total number of flights from the airports that do not contain entries in airport-codes.

In [44]:
airtraffic. \
    join(airportCodes, airtraffic["Origin"] == airportCodes["IATA_CODE"], "left"). \
    filter("IATA_CODE IS NULL"). \
    select(airtraffic["Year"], airtraffic["Month"], airtraffic["DayOfMonth"], 
           airtraffic["Origin"], airtraffic["Dest"], airtraffic["CRSDepTime"], 
           airportCodes["*"]
          ). \
    show()

+----+-----+----------+------+----+----------+---------+----+-----+-------+
|Year|Month|DayOfMonth|Origin|Dest|CRSDepTime|IATA_CODE|CITY|STATE|COUNTRY|
+----+-----+----------+------+----+----------+---------+----+-----+-------+
+----+-----+----------+------+----+----------+---------+----+-----+-------+



# * Get the total number of flights per airport that do not contain entries in airport-codes.

* This is an example for outer join.
* We need to get number of flights per airport from the 2008 January airtraffic data which do not have entries in airport-codes.
* Based on the side of the airtraffic data set, we can say left or right. We will be invoking join using airtraffic and hence we will use left outer join..
* We will be peforming join first and then we will aggregate to get number of flights from the concerned airports per airport.
* In this case will get total number of flights per airport.

In [46]:
from pyspark.sql.functions import count, col, lit

airtraffic. \
    join(airportCodes, airtraffic["Origin"] == airportCodes["IATA_CODE"], "left"). \
    filter("IATA_CODE IS NULL"). \
    groupBy("Origin"). \
    agg(count(lit(1)).alias("FlightCount")). \
    orderBy(col("FlightCount").desc()). \
    show()

+------+-----------+
|Origin|FlightCount|
+------+-----------+
+------+-----------+

